## Reading instance

In [2]:
# Reading instance

from instanceReaderSMDHPDPTW import Instance

instanceName = "AA25"

MV = 16

n_pickup_sites = 4

fleet_capacities = [10, 20, 10, 5]

vehicle_locations= [[0], [1],[2,3],[]]

inst = Instance(instanceName,
                fleet_capacities,
                n_pickup_sites,
                multiple_depots=True,
                vehicle_locations=[[0], [1],[2,3],[]])

    solver failure.


In [3]:
import numpy as np

# Applying modifications - These would be more suitable on reader!
# but for now, I'll do it like that

# Instance data
df_instance = inst.instance_data
df_instance = df_instance[['type','x','y','d','w_a','w_b']]

# Taking away artificial nodes
valid_types = ['Depot', 'Pickup','Delivery']
df_instance = df_instance[df_instance['type'].isin(valid_types)].reset_index(drop=True)

# Dropping duplicate pickup sites
df_instance = df_instance.drop_duplicates(['type','x']).reset_index(drop=True)

# Multiplying demands by factor
np.random.seed(123)
pickup_factor = 15
delivery_factor = 4
#df_instance['d'] = [demand*pickup_factor if (demand > 0) else demand*delivery_factor*np.random.randint(1,20) for demand in df_instance['d']]

df_instance['d'] = [demand*pickup_factor if (demand > 0) else demand*delivery_factor for demand in df_instance['d']]


# Appending back depot nodes at the end
df_instance = df_instance.append(df_instance[df_instance['type'] == 'Depot']).reset_index(drop=True)

In [4]:
# Distances
c = [[0 for i in range(len(df_instance))] for j in range(len(df_instance))]

for i in range(len(df_instance)):
    for j in range(len(df_instance)):
        
        x_i = df_instance.iloc[i]['x']
        x_j = df_instance.iloc[j]['x']
        y_i = df_instance.iloc[i]['y']
        y_j = df_instance.iloc[j]['y']
        
        dist = round(((x_i-x_j)**2 + (y_i-y_j)**2)**(1/2),2)
        c[i][j] = dist
        
# Demands
d = df_instance['d']

# Time windows
w_a = df_instance['w_a']
w_b = df_instance['w_b']

tw_factor = 0.8

w_b = [tw*tw_factor if tw == 1000 else tw for tw in w_b]

# Capacities
Q = fleet_capacities

In [5]:
import pandas as pd

# Importing pyomo modules

import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Creating model
model = pyo.ConcreteModel()

## Defining sets and notation

## $  N = S_{0} \cup P \cup D \cup S_{f} = \{1, ..., 3s + n\} $
## $ S_{0} = \{1, ... , s\} $
## $ P = \{s + 1, ... , 2s\} $
## $ D = \{2s + 1, ... , 2s + n\} $
## $ S_{f} = \{2s + n + 1, ... , 3s + n\} $

#### Vehicles set

## $ \newline K = \{1, ..., m\} $

#### Visits set

## $ \newline V = \{1, ..., MV\} $


In [6]:
df_instance.set_index(pd.Index(range(1,len(df_instance) + 1)))

,type,x,y,d,w_a,w_b
1,Depot,19.942,16.269,0.0,0.0,1000.0
2,Depot,39.947,22.762,0.0,0.0,1000.0
3,Depot,17.748,22.292,0.0,0.0,1000.0
4,Depot,45.989,20.442,0.0,0.0,1000.0
5,Pickup,19.942,16.269,105.0,0.0,1000.0
6,Pickup,39.947,22.762,150.0,0.0,1000.0
7,Pickup,17.748,22.292,225.0,0.0,1000.0
8,Pickup,45.989,20.442,105.0,0.0,1000.0
9,Delivery,35.573,46.733,-28.0,209.0,269.0
10,Delivery,36.158,11.897,-36.0,562.0,622.0


In [7]:
# Sets

model.setN = pyo.Set(initialize=range(len(df_instance)))
model.setP = pyo.Set(initialize=df_instance[df_instance['type'] == 'Pickup'].index)
model.setD = pyo.Set(initialize=df_instance[df_instance['type'] == 'Delivery'].index)

# Additional sets

number_of_depots = int(len(df_instance[df_instance['type'] == 'Depot'])/2)
model.setS_0 = pyo.Set(initialize=df_instance[df_instance['type'] == 'Depot'].iloc[:number_of_depots].index)
model.setS_f = pyo.Set(initialize=df_instance[df_instance['type'] == 'Depot'].iloc[-number_of_depots:].index)
model.setS = (model.setS_0 | model.setS_f)


# Set of vehicles
model.setK = pyo.Set(initialize=range(len(fleet_capacities)))

# Set of visits
model.setV = pyo.Set(initialize=range(MV))

#### Defining variables

### $x_{ijkv} = \text{1 if vehicle $k$ travels from $i$ to $j$ in its $v$ visit and 0 otherwise;} $

### $y_{ikv} = \text{amount of load in vehicle $k$ after visiting node $i$ in its $v$ visit;} $

### $w_{ikv} = \text{arrival time of vehicle $k$ in node $i$ in its $v$ visit;} $

### $z_{ikv} = \text{amount of load supplied/delivered by vehicle $k$ to node $i$ in its $v$ visit;} $

In [8]:
# Variables

model.x = pyo.Var(model.setN, model.setN, model.setK, model.setV, within = pyo.Binary)

model.y = pyo.Var(model.setN, model.setK, model.setV, within = NonNegativeReals)

model.w = pyo.Var(model.setN, model.setK, model.setV, within = NonNegativeReals)

model.z = pyo.Var(model.setN, model.setK, model.setV, within = NonNegativeReals)

#### Defining constraints

#### Objective function

## $ (1) $

## $$ max \displaystyle \sum_{i \in D}\displaystyle \sum_{i \in K}\displaystyle \sum_{j \in V}z_{ikv}$$

In [9]:
# (1) - Objective function

model.exprobj = sum(sum(sum(model.z[i,k,v] for i in model.setD) for k in model.setK) for v in model.setV) + 0.001*(sum(sum(sum(sum(model.x[i,i,k,v] for i in model.setN) for i in model.setN) for k in model.setK) for v in model.setV))

model.obj = pyo.Objective(expr= model.exprobj, sense = pyo.maximize)

## $ (2) $


## $$ \displaystyle \sum_{i \in N \setminus S_{f}}x_{ihk(v-1)} -  \displaystyle \sum_{j \in N \setminus S_{0}}x_{hjkv} = 0, \hspace{15pt} \forall h \in P \cup D, k \in K, v \in V \setminus \{0\} $$

In [10]:
# (2) - Flow conservation - if vehicle k goes from i to h, it has to go from h to j on the next visit

model.R2 = pyo.ConstraintList()

for h in (model.setP | model.setD):
    
    for k in model.setK:
        
        for v in range(1,len(model.setV)):

            model.R2.add(expr = sum(model.x[i,h,k,v-1] for i in (model.setN - model.setS_f)) - sum(model.x[h,j,k,v] for j in (model.setN - model.setS_0)) == 0)


## $ (3) $



## $$ \displaystyle \sum_{i \in N}\sum_{j \in N}x_{ijkv} \leq 1, \hspace{15pt} \forall k \in K, v \in V $$

In [11]:
# (3) There can be only one visit per visit index per vehicle

model.R3 = pyo.ConstraintList()
        
for k in model.setK:
    
    for v in model.setV:

        model.R3.add(expr = sum(sum(model.x[i, j, k, v] for i in model.setN) for j in model.setN) <= 1)

## $ (4) $



## $$ \displaystyle x_{ijk0} = 0, \hspace{15pt} \forall i \in N \setminus S_{0} , j \in N , k \in K $$

In [12]:
# (4) - Vehicles must depart from one of the available depots

model.R4 = pyo.ConstraintList()

for i in (model.setN - model.setS_0):
    
    for j in model.setN:
    
        for k in model.setK:
        
            model.R4.add(expr = model.x[i,j,k,0] == 0)

## $ (5) $



## $$ \displaystyle x_{ijkMV} = 0, \hspace{15pt} \forall j \in N \setminus S_{f} , i \in N , k \in K $$

In [13]:
# (5) - Vehicles must end their routes in one of the available depots

model.R5 = pyo.ConstraintList()

for i in model.setN:
    
    for j in (model.setN - model.setS_f):
    
        for k in model.setK:
        
            model.R5.add(expr = model.x[i,j,k,model.setV.bounds()[1]] == 0)

## $ (6) $


## $$ \displaystyle x_{ijk0} = 0,  \hspace{15pt} \forall i \in S_{0}; j \in N \setminus D_{i}; k \in K$$

In [14]:
# (6) - A vehicle can't depart from a depot that is not its initial depot

model.R6 = pyo.ConstraintList()

for i in model.setS_0:

    for j in model.setN:
        
        for k in model.setK:
            
            if (k not in vehicle_locations[i]):
                       
                model.R6.add(expr = model.x[i, j, k, 0] == 0) 

## $ (7) $

## $$ \displaystyle \sum_{j \in N}\sum_{k \in K}x_{ijk0} \leq K_{i}, \hspace{15pt} \forall i \in S_{0} $$

In [15]:
# (7) - Number of vehicles that depart from depot must be equal to number of vehicles available at depot

# K_{i} - Number of vehicles that start their route at depot i in S_0

K = [len(location) for location in vehicle_locations]

model.R7 = pyo.ConstraintList()

for i in (model.setS_0):

    model.R7.add(expr = sum(sum(model.x[i,j,k,0] for j in model.setN) for k in model.setK) <= K[i])

## $ (8) $



## $$ \displaystyle x_{ijkv} = 0, \hspace{15pt} \forall i \in S_{0}, j \in D , k \in K, v \in V $$

In [16]:
# (8) - Vehicles can't go from depot directly to a delivery node (was this supposed to be a constraint?)

model.R8 = pyo.ConstraintList()

for i in model.setS_0:
    
    for j in model.setD:
    
        for k in model.setK:
            
            for v in model.setV:
        
                model.R8.add(expr = model.x[i,j,k,v] == 0)

## $ (9) $



## $$ \displaystyle x_{ijkv} = 0, \hspace{15pt} \forall i \in N, j \in S_{f}, k \in K, v \in V \setminus {MV} $$

In [17]:
# (9) No nodes going to final depot in visits lower than MV

model.R9 = pyo.ConstraintList()

for i in model.setN:
    
    for j in model.setS_f:
        
        for k in model.setK:

            for v in range(len(model.setV) - 1):

                model.R9.add(expr = model.x[i, j, k, v] == 0)

## $ (10) $



## $$ \displaystyle x_{ijkv} = 0, \hspace{15pt} \forall i \in S_{0}, j \in N, k \in K, v \in V \setminus {0} $$

In [18]:
# (10) No nodes from initial depots in visits higher than 0

model.R10 = pyo.ConstraintList()

for i in model.setS_0:
    
    for j in model.setN:
        
        for k in model.setK:

            for v in range(1, len(model.setV)):

                model.R10.add(expr = model.x[i, j, k, v] == 0)

## $ (11) $

## $$ w_{jkv} \geq w_{ik(v-1)} + t_{ij} - M(1-x_{ijkv}), \newline\forall i \in N, j \in N, k \in K, v \in V \setminus\{0\} $$

In [ ]:
# (11) - Time windows constraints - 1

# Defining big M

M = 1000

model.R11 = pyo.ConstraintList()

for i in model.setN:
    
    for j in model.setN:
        
        for k in model.setK:
    
            for v in range(1, len(model.setV)):
        
                model.R11.add(expr = model.w[j, k, v] >= model.w[i, k, v-1] + c[i][j] - M*(1 - model.x[i,j, k, v]))
            
                model.R11.add(expr = model.w[j, k, v] <= model.w[i, k, v-1] + c[i][j] + M*(1 - model.x[i,j, k, v]))

## $ (11) $



## $$ w_{i}^{a} \leq w_{ikv} \leq w_{i}^{b} , \hspace{15pt} \forall i \in P \cup D, k \in K, v \in V $$

In [ ]:
# (11)  - Time windows constraints - 2

# model.R11 = pyo.ConstraintList()

for i in (model.setP | model.setD):
    
    for k in model.setK:
        
        for v in model.setV:
    
            model.R11.add(expr = model.w[i, k, v] >= w_a[i])
            model.R11.add(expr = model.w[i, k, v] <= w_b[i])

## $ (12) $



## $$ y_{jkv} \geq y_{ik(v-1)} + z_{jkv} - M(1-x_{ijkv}), \newline\forall i \in N, j \in P, k \in K, v \in V \setminus \{0\} $$

In [ ]:
# (12) - Load constraints - 1 (pickup)

model.R12 = pyo.ConstraintList()

for i in model.setN:
    
    for j in model.setP:
        
        for k in model.setK:
    
            for v in range(1, len(model.setV)):
        
                model.R12.add(expr = model.y[j,k,v] >= model.y[i,k,v-1] + model.z[j,k,v] - M*(1 - model.x[i,j,k,v]))
            
                model.R12.add(expr = model.y[j,k,v] <= model.y[i,k,v-1] + model.z[j,k,v] + M*(1 - model.x[i,j,k,v]))
                
# (12) - Load constraints - first pickup

for i in model.setN:
        
    for k in model.setK:

            model.R12.add(expr = model.y[i,k,0] >= model.z[i,k,0])

            model.R12.add(expr = model.y[i,k,0] <= model.z[i,k,0])

## $ (13) $



## $$ y_{jkv} \geq y_{ik(v-1)} - z_{jkv} - M(1-x_{ijkv}), \newline\forall i \in N, j \in D, k \in K, v \in V \setminus \{0\} $$

In [ ]:
# (13)  - Load constraints - 2 (delivery)

model.R13 = pyo.ConstraintList()

for i in model.setN:
    
    for j in model.setD:
        
        for k in model.setK:
    
            for v in range(1, len(model.setV)):
            
                model.R13.add(expr = model.y[j,k,v] >= model.y[i,k,v-1] - model.z[j,k,v] - M*(1 - model.x[i,j,k,v]))
                
                model.R13.add(expr = model.y[j,k,v] <= model.y[i,k,v-1] - model.z[j,k,v] + M*(1 - model.x[i,j,k,v]))

## $ (14) $



## $$ \displaystyle y_{ikv} \leq Q_{k}, \hspace{15pt} \forall i \in N , k \in K, v \in V$$

In [ ]:
# (14) - Load constraints - 3

model.R14 = pyo.ConstraintList()

for i in model.setN:

        for k in model.setK:

            for v in model.setV:

                model.R14.add(expr = model.y[i,k,v] <= Q[k])

## $ (15) $


## $$ \displaystyle \sum_{k \in K}\sum_{v \in V} z_{ikv} \leq d_{i}, \hspace{15pt} \forall i \in P \cup D $$

In [ ]:
# (15)  - Total amount picked up/delivered to each node can't be above node supply/demand

model.R15 = pyo.ConstraintList()

for i in (model.setP | model.setD):

    model.R15.add(expr = sum(sum(model.z[i, k, v] for k in model.setK) for v in model.setV) <= abs(d[i]))

## $ (16) $


## $$  z_{ikv} \leq d_{i}\displaystyle \sum_{j \in N}x_{jikv}, \hspace{15pt} \forall i \in N, k \in K, v \in V $$

In [ ]:
# (16) - Amount delivered to a node in a single visit can't exceed node demand

model.R16 = pyo.ConstraintList()

for i in model.setN:
    
    for k in model.setK:
        
        for v in model.setV:

            model.R16.add(expr = model.z[i, k, v] <= abs(d[i])*sum(model.x[j,i,k,v] for j in model.setN))

## $ (17) $


## $$  \displaystyle \sum_{i \in P}\sum_{v \in V} z_{ikv} =\displaystyle \sum_{i \in D}\sum_{v \in V}z_{ikv}, \forall k \in K$$

In [ ]:
# (17) - Total delivered amount by a vehicle needs to be equal to total picked up demand

model.R17 = pyo.ConstraintList()

for k in model.setK:
    
    model.R17.add(expr = sum(sum(model.z[i, k, v] for i in model.setP) for v in model.setV) == sum(sum(model.z[i, k, v] for i in model.setD) for v in model.setV))


## $ (18) $


## $$ M(1-x_{iikv}) \leq z_{ikv} \leq -M(1-x_{iikv}), \forall i \in N; k \in K; v \in V $$

In [ ]:
# (18) if "fake visit", no load is picked up/delivered

model.R18 = pyo.ConstraintList()

for i in model.setN:
        
    for k in model.setK:
        
        for v in model.setV:

            model.R18.add(expr = model.z[i,k,v] <= M*(1-model.x[i, i, k, v]))
            model.R18.add(expr = model.z[i,k,v] >= (-1)*M*(1-model.x[i, i, k, v]))

In [ ]:
# No arcs between pickup nodes (is this necessary?)

model.RPP = pyo.ConstraintList()

for i in model.setP:
    
    for j in model.setP:
        
        for k in model.setK:

            for v in model.setV:

                model.RPP.add(expr = model.x[i, j, k, v] == 0)

In [ ]:
# (19) - Forces that a vehicle can't go to a pickup node that is not in the same place of its initial depot

model.R22 = pyo.ConstraintList()

for i in model.setS_0:

    for j in model.setP:
        
        for k in model.setK:
            
            if (c[i][j] > 0):

                model.R22.add(expr = model.x[i, j, k, 0] == 0)

#### Valid inequalities

In [ ]:
# # Inequalities for N set

model.VI1 = pyo.ConstraintList()

# No "node to initial depot" arcs or "final depot to nodes" arcs

for i in model.setN:
    
    for k in model.setK:
        
        for v in model.setV:
    
            #model.VI1.add(expr= model.x[i, i, k, v] == 0)

            for h in model.setS_0:

                model.VI1.add(expr= model.x[i, h, k, v] == 0)

            for h in model.setS_f:

                model.VI1.add(expr= model.x[h, i, k, v] == 0)
                

# Vehicles can't visit pickup node right before ending route

model.RV2 = pyo.ConstraintList()

for i in model.setP:
    
    for j in model.setS_f:
    
        for k in model.setK:
        
            model.RV2.add(expr = model.x[i,j,k,model.setV.bounds()[1]] == 0)


# Time windows inequalities

model.VI3 = pyo.ConstraintList()

for i in (model.setP | model.setD):
    
    for j in (model.setP | model.setD):
        
        for k in model.setK:
            
            for v in model.setV:
        
                if w_a[i] + c[i][j] > w_b[j]:

                    model.VI3.add(expr= model.x[i, j, k, v] == 0)


In [ ]:
# # "Equity" constraint

# demand_fraction = 0.05

# model.REQ = pyo.ConstraintList()

# for i in model.setD:
    
#     model.REQ.add(expr= sum(sum(model.z[i,k,v] for k in model.setK) for v in model.setV) >= abs(d[i])*demand_fraction)

#### Solving model

In [ ]:
# Solver

TimeLimit = 2000

opt = pyo.SolverFactory('cplex', executable='C:/Program Files/IBM/ILOG/CPLEX_Studio201/cplex/bin/x64_win64/cplex')
opt.options['TimeLimit'] = TimeLimit


results = opt.solve(model, tee=True)

In [ ]:
for k in model.setK:
    
    for v in model.setV:

        for i in model.setN:

            for j in model.setN:

                if (pyo.value(model.x[i,j,k,v]))>0.01:

                    print(f'x[{i}, {j}, {k}, {v}] == {pyo.value(model.x[i,j,k,v])}')

In [ ]:
for k in model.setK:
    
    for v in model.setV:

        for i in model.setN:

            for j in model.setN:

                if ((pyo.value(model.x[i,j,k,v]))>0.01):

                    print(f'z[{j}, {k}, {v}] == {pyo.value(model.z[j,k,v])}')

In [ ]:
for k in model.setK:
    
    for v in model.setV:

        for i in model.setN:

            for j in model.setN:

                if ((pyo.value(model.x[i,j,k,v]))>0.01):

                    print(f'y[{j}, {k}, {v}] == {pyo.value(model.y[j,k,v])}')

In [ ]:
for k in model.setK:

    for v in model.setV:

        for i in model.setN:
            
            for j in model.setN:
            
                if pyo.value(model.x[i,j,k,v]) > 0.01:

                    print(f'w[{j}, {k}, {v}] == {pyo.value(model.w[j,k,v])}')
                    

In [ ]:
# Getting routes on solution

routes = [[] for k in model.setK]

for k in model.setK:
    
    for v in model.setV:

        for i in model.setN:

            for j in model.setN:

                if (pyo.value(model.x[i,j,k,v]))>0.01:
                    
                    if len(routes[k]) == 0:
                    
                        routes[k].extend([i,j])
                        
                    else:
                        
                        routes[k].append(j)

# print(routes)

In [ ]:
for route in routes:
    
    print('[', end="")
    
    for index_node in range(1,len(route)):
        
        if route[index_node-1] != route[index_node]:
        
            print(route[index_node-1], end=" ")
        
    print(route[index_node], end="")
    
    print(']\n')

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib, random

random.seed(123)
colors = dict(matplotlib.colors.cnames.items())

hex_colors = tuple(colors.values())


fig = px.scatter(df_instance,x="x", y="y", symbol='type', symbol_sequence=["square","star","cross", "star", "x"], color='type', text=df_instance.index, hover_data={"x":False, "y":False,  "d":True, "type":True, "index":df_instance.index})

fig.update_traces(textposition='top center')

fig.update_layout(font=dict(size=9))

# Create scatter trace of text labels

for route in routes:
    
    color = random.choice(hex_colors)
    
    for index_node in range(1, len(route)):
        
        fig.add_shape(type="line",
            x0=df_instance.iloc[route[index_node-1]].x, 
            y0=df_instance.iloc[route[index_node-1]].y,
            x1=df_instance.iloc[route[index_node]].x,
            y1=df_instance.iloc[route[index_node]].y,

            line=dict(color=color,width=3), 
            opacity = 0.2,
            visible = True
        )
        

fig.show("svg")

In [ ]:
df_instance